In [1]:
# importamos librerias varias
import pandas as pd
from multiprocessing import Pool
from pandas_ods_reader import read_ods
import glob

In [2]:
%%time
# Cargamos desde los pickles
salientes = pd.read_pickle('salientes_01.pkl')
entrantes = pd.read_pickle('entrantes_01.pkl')

CPU times: user 168 ms, sys: 192 ms, total: 359 ms
Wall time: 367 ms


In [3]:
#salientes.sample(20)

In [4]:
# sacamos un listado de telefonos y la cantidad de cada uno
resumen_llamantes = salientes['llamantes'].value_counts()
resumen_llamantes

1151467918    79226
1145635562    70662
1165409227    48861
1133294894    45722
1121765241    42413
              ...  
2291501888        1
1154181869        1
1176115172        1
1158764385        1
1121804490        1
Name: llamantes, Length: 69414, dtype: int64

In [5]:
# Convertimos lo anterior a un dataframe
# resumen_salientes = salientes.value_counts().rename_axis('telefonos').reset_index(name='llamadas')
# resumen_salientes = salientes.value_counts().rename_axis('telefonos').to_frame('cantidad')
# resumen_salientes

value_counts = salientes['llamantes'].value_counts(dropna=True, sort=True)
# solution here
df_val_counts = pd.DataFrame(value_counts)
df_value_counts_reset = df_val_counts.reset_index()
df_value_counts_reset.columns = ['llamantes', 'cantidad'] # change column names

In [6]:
df_value_counts_reset

,llamantes,cantidad
0,1151467918,79226
1,1145635562,70662
2,1165409227,48861
3,1133294894,45722
4,1121765241,42413
...,...,...
69409,2291501888,1
69410,1154181869,1
69411,1176115172,1
69412,1158764385,1


In [17]:
df_value_counts_reset['cantidad'].nunique()

1772

In [7]:
df_value_counts_reset[df_value_counts_reset['cantidad'] <= 4 ]['llamantes'].count() 

32680

In [10]:
# Filtrados en la data del hecho +- 20 minutos 07/09/2023 20;45
fil_sal_hecho = salientes.loc[(salientes['fecha-hora'] >= '2023-09-07 20:25:00')
                     & (salientes['fecha-hora'] < '2023-09-07 21:05:00')]

In [11]:
fil_sal_hecho

,llamantes,destino,llamado,fecha-hora,duracion
131337,1121674689,Tráfico incluid,295523623,2023-09-07 20:25:00,76
131338,1132489070,Conexión Móvil,440585111,2023-09-07 20:25:00,373
131339,1121885821,Tráfico incluid,820,2023-09-07 20:25:02,618
131340,1138831912,Tráfico incluid,308714838,2023-09-07 20:25:02,390
131341,1158789838,Conexión Móvil,944849233,2023-09-07 20:25:02,152
...,...,...,...,...,...
137225,1127280754,Tráfico incluid,210723099,2023-09-07 21:04:58,2979882
137226,1133439623,Tráfico incluid,290502442,2023-09-07 21:04:58,132909
137227,1171085653,Conexion Movil,857611030,2023-09-07 21:04:58,0
137228,1127280754,WhatsApp Gratis,210723098,2023-09-07 21:04:59,5132


In [9]:
# Contamos la cantidad de numeros de telefonos unicos
salientes['llamantes'].nunique()

69414

In [8]:
salientes.describe()

,llamantes,llamado,duracion
count,6.986719e+06,6.986719e+06,6.986719e+06
mean,1.204228e+09,4.400622e+10,1.127213e+07
std,3.167989e+08,6.239795e+13,1.079167e+08
min,1.120120e+09,1.000000e+00,0.000000e+00
25%,1.132265e+09,2.539629e+08,4.400000e+02
50%,1.151468e+09,2.997480e+08,2.107000e+03
75%,1.162340e+09,8.297394e+08,2.684500e+04
max,4.910007e+09,1.153214e+17,1.073747e+10


In [11]:
print(max(salientes['fecha-hora']))

2023-09-14 00:00:00


In [12]:
print(min(salientes['fecha-hora']))

2023-08-07 00:00:01


In [13]:
print(salientes['destino'].unique())

['Tráfico incluid' 'Conexion Movil' 'Conexión Móvil' 'WhatsApp Gratis'
 'TRAFICO DATOS ' 'LLAM. LOCAL' 'LLAM.LOC.MOV.' 'SERV.CLIENTE' 'ASIST.RET'
 'ADELIA MARIA' 'LLAM. 0810' 'BUENOS AIRES' '500 KB Internet' 'ROSARIO'
 'ROA volteABREV' '*555' 'COMODORO RIVADA' 'LLAM.LOCAL' 'TRAFICO DATOS'
 'GM APN2 GM Exced' 'PERGAMINO' 'SAN JOSE FELICI' 'LLAM. 0800'
 'VILLA GESELL' 'Llamada Recib' 'JUAN MARIA GUTI' 'VICTORIA'
 'LLAM.LOC.MOV' 'CORDOBA' 'PUERTO RICO' 'GENERAL MADARIA' 'CAMPANA'
 'ANSES' 'CATAMARCA' 'VILLA DOLORES' 'SALTO' 'CONCEPCION TUC.'
 'CORRIENTES' 'SAN NICOLAS' 'FRIAS' 'GM APN2 INFOT Conf' 'MAR DE AJO'
 'POSADAS' 'FORMOSA' 'PUERTO IGUAZU' 'ELDORADO' 'ALCARAZ' 'SANTA FE'
 'MAR DEL PLATA' 'GENERAL VILLEGA' 'S MIGUEL DE TUC' 'CHIVILCOY'
 'LA PLATA' 'GUALEGUAY' 'ROQUE SAENZ PEÑ' 'GOYA' 'ORAN' 'SAN JUAN'
 'LINCOLN' 'SAN ANDRES DE G' 'MENDOZA' 'SANTO TOME' 'OBERA' 'ZARATE'
 'GM APN2 GM Conf' 'SUNCHO CORRAL' 'LA RIOJA' 'SAN PEDRO JUJUY' 'LA FALDA'
 'VENADO TUERTO' '*Numeros Amigos' 'LOBO

In [26]:
#v = df.Parameter.value_counts()
salientes[salientes['llamantes'].isin(resumen_llamantes.index[resumen_llamantes.lt(2)])]

,Nro. Llamante,Destino,Nro. Llamado,fecha y hora,Duracion (Seg),Id. celda,Descripc. Celda,Latitud,Longitud,Dir Celda,fecha y hora
293,1133078828,Conexion Movil,388515017,13/09/2023 0:01:59,630609082,CF01147L200MC3,Crucero Belgrano,"-34,709811","-58,732226",Avda Mosconi 1815,2023-09-13 00:01:59
360,1167175196,Conexion Movil,419412002,13/09/2023 0:02:25,16735380,CF01212L100MC3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-09-13 00:02:25
463,1139073245,Conexion Movil,4769023,13/09/2023 0:03:33,1280277205,CF01212L100MC2,BRIGADA AÉREA,"-34,700405","-58,737688",Curapaligue 1069,2023-09-13 00:03:33
1284,1171094220,Conexion Movil,1789312018,13/09/2023 0:11:40,36227895,CF01148L100MA2,LIBERTAD 2,"-34,7097","-58,7097",PEREYRA 50,2023-09-13 00:11:40
1377,1133597670,Conexion Movil,2412913006,13/09/2023 0:12:30,16821831,CF01212L100MA3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-09-13 00:12:30
...,...,...,...,...,...,...,...,...,...,...,...
161688,1138094771,Conexion Movil,2790711004,17/08/2023 22:49:46,146412637,CF01148L200MB3,Libertad 2,"-34,712134","-58,710058",PEREYRA 50,2023-08-17 22:49:46
163753,1126809352,Conexión Móvil,27463487,17/08/2023 23:09:31,2442,CF05175U100MA1,Crucero Belgrano 3,"-34,703730","-58,724950",BEBEDERO 1992,2023-08-17 23:09:31
164501,1132543437,Conexion Movil,108808007,17/08/2023 23:17:49,749297736,CF01212L100MA3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-08-17 23:17:49
167587,1126903295,Conexion Movil,76098024,17/08/2023 23:49:10,48895474,CF02081L100MB2,CRUCERO BELGRANO 2,"-34,722076","-58,724047",RP100(Av.Patricios) entre Murillo y Azul 1003,2023-08-17 23:49:10
